In [2]:
import mne
import matplotlib.pyplot as plt
import numpy as np 
import os
from itertools import chain
from scipy.fft import fft, fftfreq

In [53]:
def process_eeg_files(folder_path):
    y_frequencies_dict = {}
    
    # Loop over all .set files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.set'):
            file_path = os.path.join(folder_path, filename)
            raw_eeg = mne.io.read_raw_eeglab(file_path, preload=True)
            
            # Preprocess the raw EEG data
            raw_eeg = preprocess_raw_eeg(raw_eeg)
            
            # Extract and modify events
            events, event_id = extract_and_modify_events(raw_eeg)
            
            # Create epochs
            epochs = create_epochs(raw_eeg, events, event_id)

            conditions = ['frequent_rare', 'rare_frequent', 'frequent_frequent']
            
            y_frequencies_dict[filename.split('.')[0]] = {}
            for condition in conditions: 

                # Separate epochs into pre and post-stimulus windows
                epochs_pre, epochs_post = separate_epochs(epochs[condition])

                assert len(epochs_pre) == len(epochs_post)
                
                # Perform FFT on the Pz channel data
                yf_avg_pre, yf_avg_post = perform_fft_on_epochs(epochs_pre, epochs_post)
                
                xf = get_xf(epochs_pre)

                assert yf_avg_pre.shape == xf.shape 

                #Compute ERP for condition
                erp = compute_erp(epochs[condition])

                #Compute FFT for ERP
                fft_erp = perform_fft_on_erp(erp)
                
                xf_erp = get_xf(erp)
                
                assert fft_erp.shape == xf_erp.shape

                # Update the dictionary with the results
                y_frequencies_dict[filename.split('.')[0]][condition] = [yf_avg_pre, yf_avg_post, fft_erp, xf, xf_erp]
    
    return y_frequencies_dict

def preprocess_raw_eeg(raw_eeg):
    """
    Preprocess the raw EEG data by setting the montage, renaming channels, and setting channel types.
    """
    # Set standard 10-20 montage
    montage = mne.channels.make_standard_montage('standard_1020')
    raw_eeg.rename_channels({"FP1": 'Fp1', 'FP2': 'Fp2'})
    raw_eeg.set_channel_types({
        'HEOG_left': 'eog', 'HEOG_right': 'eog',
        'VEOG_lower': 'eog', '(uncorr) HEOG': 'eog',
        '(uncorr) VEOG': 'eog'
    })
    raw_eeg.set_montage(montage)
    return raw_eeg

def extract_and_modify_events(raw_eeg):
    """
    Extract events from annotations and modify event IDs according to specific conditions.
    """
    events, _ = mne.events_from_annotations(raw_eeg)
    
    # Remove events with IDs 6 and 7
    events = events[~np.isin(events[:, 2], [6, 7])]
    
    # Define target and non-target event IDs
    target_ids = [1, 9, 15, 21, 27]
    nontarget_ids = [2, 3, 4, 5, 8, 10, 11, 12, 13, 14,
                     16, 17, 18, 19, 20, 22, 23, 24, 25, 26]
    
    # Modify event IDs based on conditions
    for i in range(len(events)):
        current_id = events[i, 2]
        if current_id in target_ids:
            events[i, 2] = 100  # 'frequent_rare'
        elif (i > 0 and current_id in nontarget_ids and
              events[i - 1, 2] == 100):
            events[i, 2] = 200  # 'rare_frequent'
        elif (i > 0 and current_id in nontarget_ids and
              events[i - 1, 2] in [200, 300]):
            events[i, 2] = 300  # 'frequent_frequent'
    
    # Define new event IDs
    event_id = {
        "frequent_rare": 100,
        "rare_frequent": 200,
        "frequent_frequent": 300
    }
    return events, event_id

def create_epochs(raw_eeg, events, event_id):
    """
    Create epochs from the raw EEG data using the modified events.
    """
    epochs = mne.Epochs(
        raw_eeg, events=events, event_id=event_id,
        tmin=-0.6, tmax=0.6, picks=['Pz'],
        baseline=None, preload=True
    )
    return epochs

def separate_epochs(epochs):
    """
    Separate epochs into pre-stimulus and post-stimulus windows.
    """
    epochs_pre = epochs.copy().crop(tmin=-0.6, tmax=0)
    epochs_post = epochs.copy().crop(tmin=0, tmax=0.6)
    return epochs_pre, epochs_post

def perform_fft_on_epochs(epochs_pre, epochs_post):
    """
    Perform FFT on the Pz channel data of pre and post-stimulus epochs.
    """
    # Extract Pz channel data
    pre_data = epochs_pre.copy().pick(['Pz']).get_data()
    post_data = epochs_post.copy().pick(['Pz']).get_data()
    
    # Average over epochs
    pre_data_mean = pre_data.mean(axis=0)[0]
    post_data_mean = post_data.mean(axis=0)[0]
    
    # Perform FFT
    yf_avg_pre = np.abs(fft(pre_data_mean))
    yf_avg_post = np.abs(fft(post_data_mean))
    return yf_avg_pre, yf_avg_post

def compute_erp(epochs_post): 
    erp = epochs_post.average() 
    return erp

def perform_fft_on_erp(erp):
    data = erp.pick(['Pz']).get_data()
    print(data.shape)
    erp_mean = data.mean(axis=0)
    print(erp_mean.shape)
    erp_avg =  np.abs(fft(erp_mean))
    return erp_avg


def get_xf(epoch_window):

    # Getting the sampling frequency
    sample_rate = epoch_window.info['sfreq']

    # Acessing the num of epochs, channels, and time points
    data = epoch_window.get_data()

    #N is required for xf calculations
    N = data.shape[-1]

    xf = fftfreq(N, 1/sample_rate)
    return xf

if __name__ == '__main__':
    folder_path = "C:/Users/leofl/OneDrive/Desktop/ERP Data/Project Cygnus/"
    y_frequencies_dict = process_eeg_files(folder_path)
    # Now y_frequencies_dict contains the FFT results for each file


Reading C:\Users\leofl\OneDrive\Desktop\ERP Data\Project Cygnus\p3_1.fdt


Reading 0 ... 119551  =      0.000 ...   466.996 secs...


C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:8: RuntimeWarning: Data file name in EEG.data (1_P3_shifted_ds_reref_ucbip_hpfilt_ica_weighted.fdt) is incorrect, the file name must have changed on disk, using the correct file name (p3_1.fdt).
  raw_eeg = mne.io.read_raw_eeglab(file_path, preload=True)


Used Annotations descriptions: ['11', '12', '13', '14', '15', '201', '202', '21', '22', '23', '24', '25', '31', '32', '33', '34', '35', '41', '42', '43', '44', '45', '51', '52', '53', '54', '55']
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 200 events and 309 original time points ...
0 bad epochs dropped
(1, 309)
(309,)
(1, 309)
(309,)
(1, 309)
(309,)


C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:125: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  post_data = epochs_post.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:155: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epoch_window.get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().

...

C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:125: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  post_data = epochs_post.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:155: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epoch_window.get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().

Used Annotations descriptions: ['11', '12', '13', '14', '15', '201', '202', '21', '22', '23', '24', '25', '31', '32', '33', '34', '35', '41', '42', '43', '44', '45', '51', '52', '53', '54', '55']
Not setting metadata
200 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 200 events and 309 original time points ...
0 bad epochs dropped
(1, 309)
(309,)
(1, 309)
(309,)
(1, 309)
(309,)
Reading C:\Users\leofl\OneDrive\Desktop\ERP Data\Project Cygnus\p3_9.fdt
Reading 0 ... 90367  =      0.000 ...   352.996 secs...


C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:125: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  post_data = epochs_post.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:155: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epoch_window.get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().

Used Annotations descriptions: ['11', '12', '13', '14', '15', '201', '202', '21', '22', '23', '24', '25', '31', '32', '33', '34', '35', '41', '42', '43', '44', '45', '51', '52', '53', '54', '55']
Not setting metadata
185 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 185 events and 309 original time points ...
0 bad epochs dropped
(1, 309)
(309,)
(1, 309)
(309,)
(1, 309)
(309,)


C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:125: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  post_data = epochs_post.copy().pick(['Pz']).get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:155: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epoch_window.get_data()
C:\Users\leofl\AppData\Local\Temp\ipykernel_15060\2094287080.py:124: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  pre_data = epochs_pre.copy().

In [61]:
print(len(y_frequencies_dict['p3_1']['frequent_frequent']))

5
